# * Interim : 2025 Target Sales DTAC
    Waiting for CDS developing

## Parameter

In [16]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 2, 17)

## ETL Process...

### Step 1 : Import Data Source

In [17]:
''' Rawdata '''

# Target_Sales_DTAC_Y2025
src_file = '../../data/interim/Target_Sales_DTAC_Y2025.xlsx'
src_sheet = 'Rawdata'
src_df = pd.read_excel(src_file, sheet_name=src_sheet, index_col=None) 
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'DATA_MONTH': 'MONTH_SHORT', 'DATA_YEAR': 'TM_KEY_YR', 'METRIC_VALUES': 'MTH_VALUE'}, inplace=True)
src_df['MONTH_SHORT'] = src_df['MONTH_SHORT'].apply(lambda x: x.upper())
# src_df = src_df.replace(np.nan, None)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df.tail(3)


src_df : 10088 rows, 15 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
10085,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906111.0,TRANG,H,HH,906111,TRANG,ALL
10086,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906112.0,YALA,H,HH,906112,YALA,ALL
10087,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


In [18]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)
dt_df['MONTH_SHORT'] = dt_df['MONTH_SHORT'].apply(lambda x: x.upper())
# dt_df['MONTH_KEY'] = dt_df['MONTH_SHORT'].str.upper()
# dt_df.tail(3)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['HOP_HINT']!='True Corp')]
# mooc_df.tail(3)

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)
mooc_hh_df.tail(3)

,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,AREA_KEY
2475,UPC,GX8,Regional Management 8 (South),93X,"TRANG, SATUN, PHATTHALUNG",906108.0,SATUN,906108
2476,UPC,GX7,Regional Management 7 (Central West),74Z,SAMUT SAKHON,902043.0,SAMUT SAKHON,902043
3044,UPC,GX7,Regional Management 7 (Central West),70X,"RATCHABURI, SAMUT SONGKHRAM",902044.0,SAMUT SONGKHRAM,902044


In [19]:
''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

src_df.tail(3)

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'MTH_VALUE': 'sum', 'AREA_TYPE': 'nunique', 'AREA_KEY': 'nunique'}).reset_index()
# tmp_df

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
10085,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906111.0,TRANG,H,HH,906111,TRANG,ALL
10086,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906112.0,YALA,H,HH,906112,YALA,ALL
10087,20250122,FEB,2025,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


### Step 2 : Aggregate Data

In [20]:
'''
    DB1R000900	Prepaid Inflow M1 : DTAC
    DB1S000101	Prepaid Gross Adds : DTAC
    DB2R000500	Postpaid Inflow M1 : DTAC
    DB2S000100	Postpaid Gross Adds : DTAC
'''

''' Filter '''
raw_df = src_df
raw_df = raw_df.loc[raw_df['METRIC_CD'].isin(['DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100'])]

''' Data Test '''
# raw_df = raw_df.loc[raw_df['MONTH_SHORT']=='JAN']
# raw_df = raw_df.loc[raw_df['METRIC_CD']=='DB1S000101']
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isna()]
raw_df = raw_df.reset_index(drop=True)

print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df.tail(3)


raw_df : 776 rows, 15 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
773,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,113.778035,DTAC,T,906111.0,TRANG,H,HH,906111,TRANG,ALL
774,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,20.350543,DTAC,T,906112.0,YALA,H,HH,906112,YALA,ALL
775,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,32935.095706,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


In [21]:
''' Join Area '''

merge_corp_df = raw_df.loc[raw_df['AREA_TYPE']=='True corp']
# merge_corp_df.tail(3)

merge_hh_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df.tail(3)

merge_area_df = pd.concat([merge_corp_df, merge_hh_df])
print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
merge_area_df.tail(3)

# merge_area_df = merge_hh_df
# print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
# merge_area_df.tail(3)


prep_agg_df : 776 rows, 22 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),...,AREA_KEY,AREA_DESC,CHANNEL_CD,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER
765,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,311.016564,DTAC,T,906110.0,SURAT THANI,...,906110,SURAT THANI,ALL,UPC,GX8,Regional Management 8 (South),84Z,SURAT THANI,906110.0,SURAT THANI
766,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,113.778035,DTAC,T,906111.0,TRANG,...,906111,TRANG,ALL,UPC,GX8,Regional Management 8 (South),93X,"TRANG, SATUN, PHATTHALUNG",906111.0,TRANG
767,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,20.350543,DTAC,T,906112.0,YALA,...,906112,YALA,ALL,UPC,GX8,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA


In [22]:
''' Join Period '''

merge_period_df = pd.merge(merge_area_df, dt_df, how='left', on=['TM_KEY_YR', 'MONTH_SHORT'])
# merge_period_df.tail(3)

# Prep Aggregate
prep_agg_df = merge_period_df
prep_agg_df['DAY_VALUE'] = prep_agg_df['MTH_VALUE'] / prep_agg_df['DAYS_IN_MONTH']
print(f'\nprep_agg_df : {prep_agg_df.shape[0]} rows, {prep_agg_df.shape[1]} columns')
prep_agg_df.tail(3)


prep_agg_df : 22892 rows, 27 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),...,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TM_KEY_MTH,DAY_VALUE
22889,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,20.350543,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250226,28,2025009,202502,0.726805
22890,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,20.350543,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250227,28,2025009,202502,0.726805
22891,20250122,FEB,2025,DB2S000100,Postpaid Gross Adds : DTAC,20.350543,DTAC,T,906112.0,YALA,...,Regional Management 8 (South),96X,"PATTANI, YALA, NARATHIWAT",906112.0,YALA,20250228,28,2025009,202502,0.726805


In [23]:
''' Aggregate P, G, H, HH '''

agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

# P : Nationwide
agg_p_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_G', 'TDS_SGMD']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_H', 'HOP_HINT']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_HH', 'D_CLUSTER']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
last_agg_all_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
last_agg_all_df['AGG_TYPE'] = 'S'
last_agg_all_df['FREQUENCY'] = 'Daily'
# last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'H Level 64 Province' if x>=202401 and x<=202403 else 'HH Level 96 Cluster')
last_agg_all_df['REMARK'] = 'HH Level 96 Cluster'

print(f'\nlast_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns')
# last_agg_all_df.loc[last_agg_all_df['TM_KEY_DAY']==20240601]
last_agg_all_df.tail(3)


last_agg_all_df : 39884 rows, 17 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
39881,2025,202502,2025009,20250228,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910096,SURIN,4.790306,134.128578,S,Daily,HH Level 96 Cluster
39882,2025,202502,2025009,20250228,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910097,UBON RATCHATHANI,24.645590,690.076513,S,Daily,HH Level 96 Cluster
39883,2025,202502,2025009,20250228,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910098,YASOTHON,0.000000,0.000000,S,Daily,HH Level 96 Cluster


### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [24]:
''' Input Parameter '''

# Create list
month_list = last_agg_all_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = last_agg_all_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
    AND METRIC_CD IN {v_param['metric_cd']}
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> last_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202501 AND 202502
   -> METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')

DataFrame...

   -> last_agg_all_df : 39884 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE TM_KEY_MTH BETWEEN 202501 AND 202502 
    AND METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')



In [25]:
''' Load Data '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Create rows from DataFrame
rows = [tuple(x) for x in last_agg_all_df.values]


# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
print(f'\nProcessing...')


try:
    # # Truncate
    # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

    # Delete
    cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
    # Insert
    cur.executemany(f"""
        INSERT INTO {v_target_table} 
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    cur.close()
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2025-02-17, 19:23:36

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!
